# $ \text{Т.10} $

$ \text{При снятии показаний измерительного прибора} \\ \text{ десятые доли деления шкалы прибора оцениваются "на глаз" наблюдателем.} \\ \text{ Количества цифр 0, 1, 2, ..., 9, записанных наблюдателем} \\ \text{ в качестве десятых долей при 100 независимых измерениях,} \\ \text{ равны 5, 8, 6, 12, 14, 18, 11, 6, 13, 7 соответственно.} $

### $$ \alpha = 0.05 $$

In [1]:
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.style import Style
from rich.text import Text

from scipy.special import kolmogorov
import numpy as np
import math

console = Console()
table = Table()

In [2]:
def F_norm(x, mean, sigma):
    """Функция нормального распределения."""
    return 0.5 * (1 + math.erf((x - mean)/(np.sqrt(2) * sigma)))

# Размер выборки
N = 100

m = np.array([5, 8, 6, 12, 14, 18, 11, 6, 13, 7])

## $ \text{a) Проверить гипотезу о согласии данных} \\ \text{ с законом равномерного распределения} \\ \text{ с помощью Критерия Колмогорова.} \\ \text{ Сравнить результаты с} \\ \text{ применением критерия } \chi^2 \text{.} $

$H_0: \text{все цифры равновероятны} $

$H_1: \overline H_0 $

### $ \text{По критерию согласия Колмогорова} \\ \text{ (считая распределение непрерывным):} $

$$
\tilde{\Delta} = \sqrt{n} \cdot \sup_{x \in \mathbb{R}}{|\tilde {F}(x) - F(x)|}
$$

In [3]:
F_emp = np.array([sum(m[:i]) for i in range(len(m)+1)]) / N
F_th = np.arange(10) / 10

delta = np.sqrt(N) * np.max([max(np.abs(F_th[i] - F_emp[i]),
                                 np.abs(F_th[i] - F_emp[i+1])) for i in range(F_th.size)])

p_value = kolmogorov(delta)

$$
p_{value}: P(\Delta \geq \tilde{\Delta} | H_0)  = 1 - K(\tilde{\Delta}), где
$$

$$
K(x) = 1 + 2 \sum_{k=1}^{\infty}(-1)^k \cdot e ^ {-2k^2 x^2} \text{- функция распределения Колмогорова}
$$

In [4]:
console.print(Panel(Text(f"p-value: {p_value}",
                         style="bold"),
                    title="p-value по критерию Колмогорова"),
                    justify="left")

╭─ p-value по критерию Колмогорова ─╮                                                                              
│ p-value: 0.039681879538114355     │                                                                              
╰───────────────────────────────────╯

$ p_{value} < \alpha \implies H_0 \text{ отвергается}$

$ \text{При критерии } \chi^2 \text{оснований отвергать } H_0 \text{ не было,} \\ \text{ так как } p_{value} \approx 0.059 \text{, однако более мощный} \\ \text{ критерий Колмогорова отвергает её.} $

$ \text{Можно утверждать, что критерием Колмогорова} \\ \text{ обнаружено систематическое отклонение.} $

## $ \text{b) Проверить гипотезу о согласии данных} \\ \text{ с законом нормального распределения } \\ \text{ с помощью Критерия Колмогорова.} \\ \text{ Сравнить результаты с} \\ \text{ применением критерия } \chi^2 \text{.} $

$H_0: \xi \rightsquigarrow  N(\alpha, \sigma^2) $

$H_1: \overline H_0 $

#### $ \text{(честное) Вычисление } \Delta \text{ по О.М.П.Г.} $

In [5]:
# Полуинтервалы полной группы событий
segments = np.array([(-np.inf, 1)] + [(i, i + 1) for i in range(1, 9)] + [(9, np.inf)])

# Оригинальная выборка
sample = np.repeat(np.arange(len(m)), m)

# Оценка мат. ожидания
alpha = np.mean(sample)

# Оценка корня дисперсии (несмещенная)
sigma = np.sqrt(np.var(sample) * N/(N-1))

# Функция нормального распределения по выборке
def F_norm_wave(x): return F_norm(x, alpha, sigma)

# Вероятность каждого события
P = [F_norm_wave(i[1]) - F_norm_wave(i[0]) for i in segments]

delta = sum((N * P[i] - m[i]) ** 2 / (N * P[i]) for i in range(10))

In [6]:
for i in range(-1, 9+1):
  table.add_column(f"{i}" if i >= 0 else "i")

table.add_row("m_i", *[f"{m_i}" for m_i in m])

table.add_row(*(["----"] * len(table.columns)), style=Style(color="bright_black"))

segments_formatted = [
    f"({seg[0]:.0f}; {seg[1]:.0f})" if i == 0 or i == len(segments) - 1 else f"[{seg[0]:.0f}; {seg[1]:.0f})"
    for i, seg in enumerate(segments)
]

table.add_row("П.Г.С.", *segments_formatted)

table.add_row(*(["----"] * len(table.columns)), style=Style(color="bright_black"))

table.add_row("nP_i", *[f"{(p * N):.2f}" for p in P])

console.print(table, Panel(Text(f"Δ = {delta}", style="bold"),
                    title="Δ по О.М.П.Г."), justify="left")


┏━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┓          
┃ i      ┃ 0         ┃ 1      ┃ 2      ┃ 3      ┃ 4      ┃ 5      ┃ 6      ┃ 7      ┃ 8      ┃ 9        ┃          
┡━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━┩          
│ m_i    │ 5         │ 8      │ 6      │ 12     │ 14     │ 18     │ 11     │ 6      │ 13     │ 7        │          
│ ----   │ ----      │ ----   │ ----   │ ----   │ ----   │ ----   │ ----   │ ----   │ ----   │ ----     │          
│ П.Г.С. │ (-inf; 1) │ [1; 2) │ [2; 3) │ [3; 4) │ [4; 5) │ [5; 6) │ [6; 7) │ [7; 8) │ [8; 9) │ (9; inf) │          
│ ----   │ ----      │ ----   │ ----   │ ----   │ ----   │ ----   │ ----   │ ----   │ ----   │ ----     │          
│ nP_i   │ 6.72      │ 6.85   │ 10.54  │ 13.88  │ 15.65  │ 15.10  │ 12.47  │ 8.81   │ 5.33   │ 4.65     │          
└────────┴───────────┴────────┴────────┴────────┴────────┴────────┴────────┴────────┴────────┴──────────┘          
╭──── Δ по О.М.П.Г. ─────╮                                                                                         
│ Δ = 16.871067048068763 │                                                                                         
╰────────────────────────╯

### $ \text{По критерию Колмогорова (для сложной } H_0 \text{),} \\ \text{ используя параметрический bootstrap:} $

$\vec{x}_n \rightarrow \tilde{\vec{\theta}} \text{: любой метод оценки (хотя бы состоятельный)} $

$ \xi \sim F(x, \tilde{\vec{\theta}}) $

$ M \text{ - кол-во повторений bootstrap} $

$ \text{1) } \vec{x}_n^* \rightarrow \tilde{\vec{\theta}}^* \rightarrow \\ \Delta_1^* = \sqrt{n} \sup_{x \in \mathbb{R}} | \tilde{F}^*(x) - F(x, \tilde{\vec{\theta}}^*) | $

$ ... $

$ \text{M) } \Delta_1^M = ...$

$ \text{вариационный ряд: } \Delta_{(1)}^* \dots \Delta_{(M)}^*$

$ \text{p-value:} \frac{k}{M} \text{ , где } k \text{ - число элементов: } \Delta_{(i)}^* \geq \tilde{\Delta} = \sqrt{n} \sup_{x \in \mathbb{R}} | \tilde{F}(x) - F(x, \tilde{\theta}) | $

In [7]:
# Количество повторений bootstrap
bootstrap_iterations = 50_000

In [8]:
bootstrap_delta = []

x = np.arange(10)
delta_wave = np.sqrt(N) * np.max([max(np.abs(F_norm_wave(x[i]) - F_emp[i]),
                                      np.abs(F_norm_wave(x[i]) - F_emp[i+1])) for i in range(10)])

for _ in range(bootstrap_iterations):
    # Случайная выборка, соотв норм. распределению с параметрами из О.М.П.Г.
    random_sample = np.array(sorted(np.random.normal(alpha, sigma, N)))

    # Оценка мат. ожидания по случайной подвыборке (bootstrap)
    alpha_bootstrap = random_sample.mean()

    # Оценка корня дисперсии по случайной подвыборке (bootstrap)
    sigma_bootstrap = np.sqrt(random_sample.var() * N/(N-1))

    F_bootstrap_emp = [i / N for i in range(N + 1)]

    # Функция нормального распределения по случайной подвыборке (bootstrap)
    def F_bootstrap_wave(j):
      return F_norm(random_sample[j], alpha_bootstrap, sigma_bootstrap)
    
    sup = np.sqrt(N) * np.max([max(np.abs(F_bootstrap_wave(j) - F_bootstrap_emp[j]), \
                                     np.abs(F_bootstrap_wave(j) - F_bootstrap_emp[j+1])) \
                                        for j in range(len(random_sample))])

    bootstrap_delta.append(sup)

bootstrap_delta = np.array(bootstrap_delta)

p_value = len(bootstrap_delta[bootstrap_delta >= delta_wave]) / bootstrap_iterations

In [9]:
console.print(Panel(Text(f"p-value: {p_value}",
                         style="bold"),
                    title="p-value по критерию Колмогорова"),
                    justify="left")

╭─ p-value по критерию Колмогорова ─╮                                                                              
│ p-value: 0.01518                  │                                                                              
╰───────────────────────────────────╯

$ p_{value} < \alpha \implies H_0 \text{ отвергается}$

$ \text{При критерии } \chi^2 \text{ } H_0 \text{ также отвергалась, так как } \\ p_{value} \approx 0.01825 \text{ и более мощный критерий Колмогорова } \\ \text{ отвергает её с меньшей мерой значимости.} $